In [1]:
import tabula 
import pandas as pd
from tika import parser
import os
import copy
import urllib

In [2]:
def parsing_table(file, df_all1, df_all2, df_all3, dai):  
#     if not os.path.isfile(file):
    try:
        raw = parser.from_file(file)
#     except urllib.error.HTTPError:
    except FileNotFoundError:
        skipped.append(dai)
        return df_all1, df_all2, df_all3
    if raw['status'] == 200 and raw['content'] is not None and len(raw['content'].strip().split()) > 200 and raw['content'].strip().split()[4] == '№' and raw['content'].strip().split()[5][0] == 'D':
        df = tabula.read_pdf(file, pages=1, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) == 2:
            df[0]['decl_num'] = raw['content'].strip().split()[5]
            df[1]['decl_num'] = raw['content'].strip().split()[5]
        else:
            print('akusi', dai)
        if df_all1 is None:
            df_all1 = df[0]
            df_all2 = df[1]
        else:
            df_all1 = pd.concat([df_all1, df[0]], ignore_index=True)
            df_all2 = pd.concat([df_all2, df[1]], ignore_index=True)
        df = tabula.read_pdf(file, pages=2, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) >= 1:
            df = [j for j in df if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
        for i in range(3, int(raw['metadata']['xmpTPg:NPages']) + 1):
            f2 = tabula.read_pdf(file,
                         pages=i, pandas_options={'header': None}, spreadsheet=True, multiple_tables=True)
            if type(f2) == list and len(f2) >= 1:
                f2 = [j for j in f2 if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
            if type(f2) != list:
                df = pd.concat([df, f2], ignore_index=True)
        df['decl_num'] = raw['content'].strip().split()[5]
        if df_all3 is None:
            df_all3 = df
        else:
            df_all3 = pd.concat([df_all3, df], ignore_index=True)
    return df_all1, df_all2, df_all3

In [3]:
def data_cleaning(dat1, dat2, dat3):
#     print(path_to, file_num, 2)
    dat3 = dat3.loc[(dat3[2].isnull() == False) & (dat3[4].isnull() == False) & (dat3[2] != '3' ) & (dat3[4] != 'Клас\rнебезпеки')]
    for i in range(12):
        dat3[i] = dat3[i].apply(lambda x: str(x).replace('\r', ' '))
    dat3.columns = ['No','trash_name','code','alter_name','class','storage','value',
                                 'value_prediction','destination','value_destination','operation','operation_desc', 'decl_num']

    dat1[0] = dat1[0].apply(lambda x: str(x).replace('\r', ' '))
    dat1[1] = dat1[1].apply(lambda x: str(x).replace('\r', ' '))
    
    print(dat2[dat2[0].isnull()][dat2[dat2[0].isnull()][5] != '(x1)'], "AAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    for i in range(6):
        dat2[i] = dat2[i].apply(lambda x: str(x).replace('\r', ' '))
    dat2 = dat2.loc[(dat2[0] != 'Період утворення відходів') & (dat2[5] != '(x1)')]
    dat2.columns = ['waste_creation_time','index_gen_wastes','amount_I','amount_II', 'amount_III', 'amount_IV', 'decl_num']
    return dat1, dat2, dat3

In [4]:
def data_saving(datas, file_num):
    datas[0], datas[1], datas[2] = data_cleaning(datas[0], datas[1], datas[2])
    for j in range(3):
        datas[j].to_csv(path_to + files[j] + str(file_num) + ".csv", sep=';', index=False)
#     file_num += 1000
#         if not os.path.isfile(path_to + files[j]):
#             datas[j].to_csv(path_to + files[j], sep=';', index=False)
#         else:
#             with open(path_to + files[j], 'a') as f:
#                 datas[j].to_csv(path_to + files[j], sep=';', index=False, header=False)
    return [None, None, None]

In [ ]:
datas = [None, None, None]
# path_from = 'https://e-eco.gov.ua/parlor/applications/viewfile/id/'
path_from = './declarations/'
path_to = "./dec/data/d2/"
file_num = 42000
files = ["F", "S", "T"]
skipped = []
min_range = 41001
max_range = 43001

for i in range(min_range, max_range):
    if i % 100 == 0:
        print('--', i)
    datas[0], datas[1], datas[2] = parsing_table(path_from + str(i), datas[0], datas[1], datas[2], i)
    if i % 1000 == 0:
        datas = data_saving(datas, file_num)
        file_num += 1000
        
datas = data_saving(datas, 0)

In [ ]:
#join files into one csv file
for f in files:
    data1 = pd.read_csv("./dec/data/d2/" + f + "1000.csv", sep=';')
    for i in range(2000,43000,1000):
        data2 = pd.read_csv("./dec/data/d2/" + f + str(i) + ".csv", sep=';')
        data1 = pd.concat([data1,data2], ignore_index=True)
    if f == 'F':
        data1.to_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';', index=False)
    elif f == 'S':
        data1.to_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';', index=False)
    elif f == 'T':
        data1.to_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';', index=False)